In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

class CBAM(nn.Module):
    def __init__(self, channels, reduction_ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

        self.mlp = nn.Sequential(
            nn.Linear(channels, channels // reduction_ratio, bias=False),
            nn.ReLU(),
            nn.Linear(channels // reduction_ratio, channels, bias=False)
        )

        self.spatial = nn.Conv2d(2, 1, kernel_size=kernel_size, padding=kernel_size // 2)

    def forward(self, x):
        b, c, _, _ = x.size()
        avg_out = self.mlp(self.avg_pool(x).view(b, c))
        max_out = self.mlp(self.max_pool(x).view(b, c))
        scale = torch.sigmoid(avg_out + max_out).view(b, c, 1, 1)
        x = x * scale

        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = x * torch.sigmoid(self.spatial(torch.cat([avg_out, max_out], dim=1)))
        return x

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction),
            nn.ReLU(),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


In [ ]:
class EmotionNet(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionNet, self).__init__()
        self.residual_block = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.ReLU(), nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding=1), nn.ReLU(), nn.BatchNorm2d(64),
            nn.MaxPool2d(2)
        )
        self.attention_block = nn.Sequential(
            CBAM(3),
            nn.Conv2d(3, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.dilated_block = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, dilation=2, padding=2), nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, dilation=2, padding=2), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.efficientnet = models.efficientnet_b0(pretrained=True)
        self.efficientnet.classifier = nn.Identity()

        self.fc = nn.Sequential(
            nn.Linear(1536, 512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, 256), nn.ReLU(), nn.Dropout(0.4),
            nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.4),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x1 = self.residual_block(x)
        x1 = nn.AdaptiveAvgPool2d(1)(x1).view(x1.size(0), -1)

        x2 = self.attention_block(x)
        x2 = nn.AdaptiveAvgPool2d(1)(x2).view(x2.size(0), -1)

        x3 = self.dilated_block(x)
        x3 = nn.AdaptiveAvgPool2d(1)(x3).view(x3.size(0), -1)

        x4 = self.efficientnet(x)

        x = torch.cat((x1, x2, x3, x4), dim=1)
        return self.fc(x)

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm 

def train_model(data_dir='fer2013', batch_size=32, num_epochs=50):
    print("Training function called...")

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3)
    ])

    train_path = os.path.join(data_dir, 'train')
    val_path = os.path.join(data_dir, 'test')

    train_dataset = datasets.ImageFolder(train_path, transform=transform)
    val_dataset = datasets.ImageFolder(val_path, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    model = EmotionNet(num_classes=len(train_dataset.classes)).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()

    print("Starting training loop... ")
    best_acc = 0.0

    for epoch in range(num_epochs):
        start_time = time.time()
        print(f"\nEpoch [{epoch + 1}/{num_epochs}]")

        model.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, desc="Training", leave=False)
        for i, (images, labels) in enumerate(train_bar):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if (i + 1) % 10 == 0:
                train_bar.set_postfix(loss=loss.item())

        avg_train_loss = running_loss / len(train_loader)
        print(f"Training Loss: {avg_train_loss:.4f}")

        model.eval()
        correct = 0
        total = 0
        val_bar = tqdm(val_loader, desc="Validating", leave=False)
        with torch.no_grad():
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = 100 * correct / total
        print(f"Validation Accuracy: {acc:.2f}%")

        if acc > best_acc:
            best_acc = acc
            torch.save(model, "emotion_model_v1.pth")
            print(f"New best model saved with accuracy: {best_acc:.2f}%")

        epoch_time = time.time() - start_time
        print(f"Epoch time: {epoch_time:.2f} seconds")

    torch.save(model, "emotion_model_v1.pth")
    print("\nTraining complete. Final model saved as 'emotion_model_SQ.pth'")

In [ ]:
train_model(data_dir='./fer2013', batch_size=32, num_epochs=100)